In [1]:
import numpy as np
import pandas as pd
import matplotlib as mt
import itertools as it
import Funzioni as fn

In [2]:
ds=pd.read_csv("C:/Users/Gulli/Desktop/Uni/Magi/MachineLearningandstat/Progetto_ML/your_dataset.csv")
X=ds.iloc[:,:-1]
Y=ds.iloc[:,-1]

In [ ]:
print(ds.isnull().sum())
ds = ds.dropna()

In [3]:
#pui scrivere funzione
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
# Filtra i dati rimuovendo gli outlier
X_cleaned = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]
Y_cleaned=Y[X_cleaned.index]


In [ ]:
count1=np.sum(Y_cleaned==1)
count_inv=np.sum(Y_cleaned==-1)
print(f'{count1}, {count_inv},{Y_cleaned.shape-count_inv}')

In [4]:
X_cleaned.reset_index(drop=True, inplace=True)
Y_cleaned.reset_index(drop=True, inplace=True)

In [5]:
train = X_cleaned.sample(frac=0.7, random_state=42)  # Estrae il 70% dei dati in modo casuale
test = X_cleaned.drop(train.index)
X_test=test.values
X_train=train.values
Y_train=Y_cleaned.loc[train.index].values
Y_test=Y_cleaned.loc[test.index].values

In [6]:
#puoi scrivere funzione
train_mean = X_train.mean(axis=0)
train_std = X_train.std(axis=0)
# Standardizza il training set
X_train_standardized = (X_train - train_mean) / train_std
# Usa la stessa media e std per standardizzare il test set
X_test_standardized = (X_test - train_mean) / train_std

In [ ]:
#TUNING
perceptron_param = {
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

pegasos_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

logistic_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

In [ ]:
loss_perc_in, param_perc_in = fn.tuning_par(fn.Perceptron, pd.DataFrame(X_train_standardized), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos_in, param_pegasos_in = fn.tuning_par(fn.PegasosSVM, pd.DataFrame(X_train_standardized), pd.Series(Y_train), pegasos_param, 5)
loss_logit_in, param_logit_in = fn.tuning_par(fn.RegLogisticClass, pd.DataFrame(X_train_standardized), pd.Series(Y_train), logistic_param, 5)

In [ ]:
print(param_perc_in, loss_perc_in)
print(param_pegasos_in, loss_pegasos_in)
print(param_logit_in, loss_logit_in)

In [ ]:
correlation_matrix = X_cleaned.corr()
correlation_matrix

In [ ]:
high_corr = correlation_matrix[(correlation_matrix > 0.9) | (correlation_matrix < -0.9)]
high_corr = high_corr[high_corr != 1.0]
high_corr.stack().reset_index()

In [ ]:
#puoi scrivere funzione
np.random.seed(42)

for i in [2,5,9,[5,9],[2,5],[2,9],[2,5,9]]:
    X_reduced_train=np.delete(X_train_standardized, i, axis=1)
    X_reduced_test=np.delete(X_test_standardized, i, axis=1)
    perceptron_model = fn.Perceptron(0.1,500)
    
    perceptron_model.fit(X_reduced_train, Y_train)
    
    y_pred_perceptron_tr = perceptron_model.predict(X_reduced_train)
    y_pred_perceptron_test = perceptron_model.predict(X_reduced_test)
   
    loss_perceptron_tr = fn.Zero_One_Loss(Y_train, y_pred_perceptron_tr,Y_train.shape[0])
    loss_perceptron_test = fn.Zero_One_Loss(Y_test, y_pred_perceptron_test,Y_test.shape[0])
    print(f"0-1 Loss Perceptron (Training): {loss_perceptron_tr}  ")
    print(f"0-1 Loss Perceptron (Test): {loss_perceptron_test}  ") 

In [7]:
#togliamo gli stessi per tutti!
X_train=np.delete(X_train_standardized, [5,9], axis=1)
X_test=np.delete(X_test_standardized, [5,9], axis=1) #per logistic è la peggiore(però test mejo di training), per pegasos migliore, perceptron  migliore
#X_train_pegasos=np.delete(X_train, 2, axis=1)
#X_test_pegasos=np.delete(X_test, 2, axis=1)

In [ ]:
loss_perc, param_perc = fn.tuning_par(fn.Perceptron, pd.DataFrame(X_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos, param_pegasos = fn.tuning_par(fn.PegasosSVM, pd.DataFrame(X_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit, param_logit = fn.tuning_par(fn.RegLogisticClass, pd.DataFrame(X_train), pd.Series(Y_train), logistic_param, 5)

In [ ]:
print(param_perc)
print(param_pegasos)
print(param_logit)


METTI COME INPUT COLUMN NAMES CHE SE NO FA X1-X8 E NON TOGLIE LE VARIABILE CHE ABBIAMO TOLTO 

In [8]:
column_names = [f'x{i+1}' for i in range(X.shape[1])]
column_names=np.delete(column_names, 5)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']
['x1' 'x2' 'x3' 'x4' 'x5' 'x7' 'x8' 'x9']


In [9]:
X_poly_train, X_poly_test = fn.Polynomial_exp(X_train,X_test, column_names)
#X_poly_pega_train, X_poly_pega_test = fn.Polynomial_exp(X_train_pegasos,X_test_pegasos) , per semplicità uso stesso dataset per tutti e 3


DEVO FARE TUNING DI NUOVO PER TUTTI CON POL EXP


In [ ]:
loss_pegasos, param_pegasos = fn.tuning_par(fn.RegLogisticClass, X_poly_train, pd.Series(Y_train), pegasos_param, 5)

In [ ]:
param_pegasos #cambia tra X_train e X_poly_train

RINORMALIZZA I DATASET


In [14]:
train_mean_poly = X_poly_train.mean(axis=0)
train_std_poly = X_poly_train.std(axis=0)
# Standardizza il training set
X_train_poly_standardized = (X_poly_train - train_mean_poly) / train_std_poly
# Usa la stessa media e std per standardizzare il test set
X_test_poly_standardized = (X_poly_test - train_mean_poly) / train_std_poly

In [15]:
perceptron_model=fn.Perceptron(0.1,50)
pegasos_model=fn.PegasosSVM(0.01,500)
logistic_model=fn.RegLogisticClass(1,100)
perceptron_model.fit(X_train_poly_standardized.values, Y_train)
pegasos_model.fit(X_train_poly_standardized.values, Y_train)
logistic_model.fit(X_train_poly_standardized.values, Y_train)

In [16]:
weights_perceptron = perceptron_model.weights
weights_pegasos = pegasos_model.weights
weights_logistic = logistic_model.weights

In [17]:
# Create a DataFrame to store and compare the weights
weights_df = pd.DataFrame({
    'Feature': X_train_poly_standardized.columns,
    'Perceptron': weights_perceptron,
    'Pegasos': weights_pegasos,
    'Logistic_Regression': weights_logistic
})

In [1]:
correlation_matrix = X_train_poly_standardized.corr()
correlation_matrix

NameError: name 'X_train_poly_standardized' is not defined

In [ ]:
high_corr = correlation_matrix[(correlation_matrix > 0.9) | (correlation_matrix < -0.9)]
high_corr = high_corr[high_corr != 1.0]
high_corr.stack().reset_index()

In [ ]:
pd.set_option('display.max_rows', None)  # Mostra tutte le righe
pd.set_option('display.max_columns', None)  # Mostra tutte le colonne
pd.set_option('display.width', None)  # Adatta la larghezza alla console
pd.set_option('display.max_colwidth', None)  # Mostra interamente il contenuto delle celle

# Ora puoi visualizzare weights_df nella sua interezza
print(weights_df)

# Ripristina le opzioni dopo la visualizzazione se desideri tornare alla modalità predefinita
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

Procedura per Calcolare l'Importanza Relativa dei Pesi
-Calcolare la somma assoluta dei pesi per ciascun modello.
-Dividere ogni peso per la somma assoluta per ottenere la sua importanza relativa.
-Convertire in percentuale per rendere più comprensibile l'importanza di ciascuna feature.

In [ ]:
# Calcolo dell'importanza relativa per ogni modello
for model in ['Perceptron', 'Pegasos', 'Logistic_Regression']:
    # Calcola la somma assoluta dei pesi per il modello
    total_weight = weights_df[model].abs().sum()
    
    # Calcola l'importanza relativa dei pesi come percentuale
    weights_df[f'{model}_importance'] = (weights_df[model].abs() / total_weight) * 100

# Visualizza le importanze relative
print(weights_df[['Feature', 'Perceptron_importance', 'Pegasos_importance', 'Logistic_Regression_importance']])